In [ ]:
!pip install git+https://github.com/tensorflow/examples.git


  Cloning https://github.com/tensorflow/examples.git to /tmp/pip-req-build-ja5i1ye3
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/examples.git /tmp/pip-req-build-ja5i1ye3
  Resolved https://github.com/tensorflow/examples.git to commit fff4bcda7201645a1efaea4534403daf5fc03d42
  Preparing metadata (setup.py) ... done


In [ ]:
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix

import os
import cv2
import numpy as np
from PIL import Image

AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# special need for Google Colab
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/DocDenoise")
!ls


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
adaptive_threshold.m  denoising_cg.ipynb  README.md		   training_ae.ipynb
checkpoints	      easy_ocr.ipynb	  skew_correction.ipynb    training_cg.ipynb
data		      images		  table_recognition.ipynb  zip
denoising_ae.ipynb    paddle_ocr.ipynb	  tesseract_ocr.ipynb


In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
path_zip = 'zip/'
path = 'data/'

In [ ]:
import zipfile


In [ ]:
# unzip files first to working directory
with zipfile.ZipFile(path_zip + 'train.zip', 'r') as zip_ref:
    zip_ref.extractall(path)

with zipfile.ZipFile(path_zip + 'test.zip', 'r') as zip_ref:
    zip_ref.extractall(path)

with zipfile.ZipFile(path_zip + 'train_cleaned.zip', 'r') as zip_ref:
    zip_ref.extractall(path)

In [ ]:
train_img = sorted(os.listdir(path + 'train/'))
train_cleaned_img = sorted(os.listdir(path + 'train_cleaned/'))
test_img = sorted(os.listdir(path + 'test/'))
#test_invoice_img = sorted(os.listdir(path + 'test_invoice/'))

In [ ]:
IMG_WIDTH = 512
IMG_HEIGHT = 512

# prepare function
def process_image(path):
    img = cv2.imread(path)
    img = np.asarray(img, dtype="float32")
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = img/255.0

    return img

In [ ]:
train = []
train_cleaned = []
test = []
test_invoice = []

for f in train_img:
    train.append(process_image(path + 'train/' + f))

for f in train_cleaned_img:
    train_cleaned.append(process_image(path + 'train_cleaned/' + f))

for f in test_img:
    test.append(process_image(path + 'test/' + f))

#for f in test_invoice_img:
 #   test_invoice.append(process_image(path + 'test_invoice/' + f))

In [ ]:
# convert list to numpy array
X_train = np.asarray(train)
Y_train = np.asarray(train_cleaned)
X_test = np.asarray(test)
X_test_invoice = np.asarray(test_invoice)

In [ ]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

In [ ]:
LAMBDA = 10


In [ ]:
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)


In [ ]:
def discriminator_loss(real, generated):
  real_loss = loss_obj(tf.ones_like(real), real)

  generated_loss = loss_obj(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

In [1]:
def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

In [ ]:
def calc_cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))

  return LAMBDA * loss1

In [ ]:
def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return LAMBDA * 0.5 * loss


In [ ]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)


In [ ]:
checkpoint_path = "./checkpoints/cycleGAN"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')
else:
  print('no')

no


In [ ]:
def generate_images(model, test_input, epoch):
  prediction = model(test_input)

  plt.figure(figsize=(12, 12))

  display_list = [test_input[0], prediction[0]]
  title = ['Input Image', 'Predicted Image']

  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    plt.imshow(display_list[i])
    plt.axis('off')

  plt.savefig('images/'+str(epoch)+'.png')
  plt.show()

In [ ]:
@tf.function
def train_step(real_x, real_y):
  # persistent is set to True because the tape is used more than
  # once to calculate the gradients.
  with tf.GradientTape(persistent=True) as tape:
    # Generator G translates X -> Y
    # Generator F translates Y -> X.

    fake_y = generator_g(real_x, training=True)
    cycled_x = generator_f(fake_y, training=True)

    fake_x = generator_f(real_y, training=True)
    cycled_y = generator_g(fake_x, training=True)

    # same_x and same_y are used for identity loss.
    same_x = generator_f(real_x, training=True)
    same_y = generator_g(real_y, training=True)

    disc_real_x = discriminator_x(real_x, training=True)
    disc_real_y = discriminator_y(real_y, training=True)

    disc_fake_x = discriminator_x(fake_x, training=True)
    disc_fake_y = discriminator_y(fake_y, training=True)

    # calculate the loss
    gen_g_loss = generator_loss(disc_fake_y)
    gen_f_loss = generator_loss(disc_fake_x)

    total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)

    # Total generator loss = adversarial loss + cycle loss
    total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
    total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

    disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
    disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)

  # Calculate the gradients for generator and discriminator
  generator_g_gradients = tape.gradient(total_gen_g_loss, generator_g.trainable_variables)
  generator_f_gradients = tape.gradient(total_gen_f_loss, generator_f.trainable_variables)

  discriminator_x_gradients = tape.gradient(disc_x_loss, discriminator_x.trainable_variables)
  discriminator_y_gradients = tape.gradient(disc_y_loss, discriminator_y.trainable_variables)

  # Apply the gradients to the optimizer
  generator_g_optimizer.apply_gradients(zip(generator_g_gradients, generator_g.trainable_variables))
  generator_f_optimizer.apply_gradients(zip(generator_f_gradients, generator_f.trainable_variables))

  discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients, discriminator_x.trainable_variables))
  discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients, discriminator_y.trainable_variables))

In [ ]:
import time

In [ ]:
epochs = 1
batch_size = 6

def batch_generator(X_train, Y_train, batch_size):
    for i in range(0, len(X_train), batch_size):
        yield X_train[i:i+batch_size], Y_train[i:i+batch_size]

for epoch in range(epochs):
    start = time.time()

    for image_x, image_y in batch_generator(X_train, Y_train, batch_size):
        train_step(image_x, image_y)
        print('.', end='')

    clear_output(wait=True)

    # Using a consistent image (sample_horse) so that the progress of the model
    # is clearly visible.
    generate_images(generator_g, X_test[0].reshape(1, 512, 512, 3), epoch + 1)
    generate_images(generator_f, Y_train[0].reshape(1, 512, 512, 3), -epoch - 1)

    if (epoch + 1) % 10 == 0:
        ckpt_save_path = ckpt_manager.save()
        print('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))

    print('Time taken for epoch {} is {} sec\n'.format(epoch+1, time.time()-start))


In [ ]:
import gc  # Import the garbage collector module

epochs = 1
batch_size = 6

def batch_generator(X_train, Y_train, batch_size):
    for i in range(0, len(X_train), batch_size):
        yield X_train[i:i+batch_size], Y_train[i:i+batch_size]

for epoch in range(epochs):
    start = time.time()

    for image_x, image_y in batch_generator(X_train, Y_train, batch_size):
        train_step(image_x, image_y)
        print('.', end='')

    clear_output(wait=True)

    # Using a consistent image (sample_horse) so that the progress of the model
    # is clearly visible.
    generate_images(generator_g, X_test[0].reshape(1, 512, 512, 3), epoch + 1)
    generate_images(generator_f, Y_train[0].reshape(1, 512, 512, 3), -epoch - 1)

    if (epoch + 1) % 10 == 0:
        ckpt_save_path = ckpt_manager.save()
        print('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))

    print('Time taken for epoch {} is {} sec\n'.format(epoch+1, time.time()-start))

    # Explicitly release memory using garbage collector
    gc.collect()
    tf.keras.backend.clear_session()


In [ ]:
for inp in X_test[:4]:
  generate_images(generator_g, inp.reshape(1,512,512,3), 0)
